In [1]:
import re
import zipfile
import csv
import pandas as pd
from random import shuffle
from math import exp, log
from collections import defaultdict, Counter
from typing import List, Set, Tuple, TypedDict

In [2]:
# Unzip dataset
with zipfile.ZipFile("data/dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("data/")

In [3]:
class Message(TypedDict):
    text: str
    spam: bytes

In [4]:
messages: List[Message] = []
msgs: List[str] = []
spams: List[int] = []

def transform(dict_, typed_dict) -> dict:
    fields = typed_dict.__annotations__

    for name, value in dict_.items():
        if name == "text":
            if value == "":
                pass
            else:
                msgs.append(value.lstrip())
                
        elif name == "spam":
            if value == "":
                pass
            else:
                spams.append(value)

In [5]:
# read dataset and clean the dataset
data = "data/emails.csv"

with open(data, encoding="utf8", newline='') as file:
    for i, row in enumerate(csv.DictReader(file), 1):
        transform(row, Message)

    messages = list(zip(msgs, spams))

In [6]:
# normalize input string and extract words with length >= 2
def tokenize(text: str) -> Set[str]:
    words: List[str] = []
    for word in re.findall(r'[A-Za-z0-9\']+', text):
        if len(word) >= 3:
            words.append(word.lower())
    return set(words)

assert tokenize('Is this a text? If so, Tokenize this text!...') == {'this', 'text', 'tokenize'}

In [7]:
# SPlit dataset into train and test sets (80/20)
def train_test_split(messages: List[Message], pct=0.8) -> Tuple[List[Message], List[Message]]:
    shuffle(messages)
    num_train = int(round(len(messages) * pct, 0))
    return messages[:num_train], messages[num_train:]

In [8]:
class NaiveBayes:
    def __init__(self) -> None:
        # `k` is the smoothening factor
        self._k: int = 1
        self._num_spam_messages: int = 0
        self._num_ham_messages: int = 0
        self._num_word_in_spam: Dict[int] = defaultdict(int)
        self._num_word_in_ham: Dict[int] = defaultdict(int)
        self._spam_words: Set[str] = set()
        self._ham_words: Set[str] = set()
        self._words: Set[str] = set()

    # Iterate through the given messages and gather the necessary statistics
    def train(self, messages: List[Message]) -> None:
        msg: Message
        token: str
        for msg in messages:
            tokens: Set[str] = tokenize(msg[0])
            self._words.update(tokens)
            if msg[1] == '1':
                self._num_spam_messages += 1
                self._spam_words.update(tokens)
                for token in tokens:
                    self._num_word_in_spam[token] += 1
            else:
                self._num_ham_messages += 1
                self._ham_words.update(tokens)
                for token in tokens:
                    self._num_word_in_ham[token] += 1                
    
    # Probability of a word being spam
    def word_spam_percent(self, word: str) -> float:
        return (self._k + self._num_word_in_spam[word]) / ((2 * self._k) + self._num_spam_messages)
    
    # Probability of a word being ham
    def word_ham_percent(self, word: str) -> float:
        return (self._k + self._num_word_in_ham[word]) / ((2 * self._k) + self._num_ham_messages)
    
    # predict a word if it's spam
    def predict(self, text: str) -> float:
        text_words: Set[str] = tokenize(text)
        log_p_spam: float = 0.0
        log_p_ham: float = 0.0

        for word in self._words:
            p_spam: float = self.word_spam_percent(word)
            p_ham: float = self.word_ham_percent(word)
            if word in text_words:
                log_p_spam += log(p_spam)
                log_p_ham += log(p_ham)
            else:
                log_p_spam += log(1 - p_spam)
                log_p_ham += log(1 - p_ham)

        if_spam: float = exp(log_p_spam)
        if_ham: float = exp(log_p_ham)
        sum = if_spam + if_ham
        if sum == 0:
            sum = 0.1

        return round(if_spam / sum, 6)


In [9]:
train: List[Message]
test: List[Message]

train, test = train_test_split(messages)
print(len(train), len(test))

4581 1145


In [10]:
nb = NaiveBayes()
nb.train(train)

print(f'Spam messages in training data: {nb._num_spam_messages}')
print(f'Ham messages in training data: {nb._num_ham_messages}')
print(f'Most spam words: {Counter(nb._num_word_in_spam).most_common(20)}')

Spam messages in training data: 1107
Ham messages in training data: 3474
Most spam words: [('subject', 1107), ('the', 869), ('and', 803), ('you', 793), ('your', 775), ('for', 731), ('this', 540), ('with', 520), ('have', 509), ('our', 498), ('not', 495), ('are', 479), ('from', 468), ('here', 428), ('that', 422), ('will', 387), ('all', 382), ('com', 357), ('more', 329), ('now', 308)]


In [11]:
# Manual test
# spam: 1, 3
emails = [
    "Hey, Jill!\nIt's great to have you as part of our 8 Ball Pool family!\nHere's a quick tip we hope you'll find useful:\nPlay using Facebook login & reap rewards like...\n5 FREE Pool Cash at time of login\nFree Coins every hour\nPlay on multiple devices\nPlay with and challenge your friends\nFree Gifts\nHere's a little gift to help you along your 8 Ball Pool journey. Click the button below to collect now!",
    "It was nice meeting you earlier. It would be great to be able to see you again.",
    "Hello\nAre you tired? Are you exhausted after a long day at work? Signup to receive a free spa at testurl.com"
]

for email in emails:
    print(f"Probability: {nb.predict(email)}")

Probability: 0.999052
Probability: 1e-06
Probability: 0.894392


In [12]:
# Predict messages from test set
spam_prob: float = 0.0
spam_count: int = 0
spam_predict_count: int = 0

for msg in test:
    prob: float = nb.predict(msg[0])

    # Count spam messages
    if(msg[1] == "1"):
        spam_count += 1

    # Return the prediction of all the test message

    if(prob >= 0.4):
        spam_prob += prob
        spam_predict_count += 1

print(f"spam count: {spam_count}")
print(f"spam predicted: {spam_predict_count}")
print(f"Accuracy of spam prediction: {(spam_prob / spam_count) * 100}\n")


spam count: 261
spam predicted: 218
Accuracy of spam prediction: 82.49105938697319

